In [ ]:
!pip install flask flask-ngrok joblib

In [ ]:
!pip install category_encoders

In [ ]:
!ngrok authtoken 2gY7mcXdRpOKQFk85B7xq0_5V63PccLiVuxgbAqZeBGB


In [ ]:
!pip install flask-ngrok pyngrok joblib pandas flask

from flask import Flask, request, jsonify
from pyngrok import ngrok
import joblib
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Initialize Flask app
app = Flask(__name__)

# Load the trained model and encoder
model = joblib.load('/content/drive/My Drive/dt_model.joblib')
encoder = joblib.load('/content/drive/My Drive/loo_encoder.joblib')

@app.route('/')
def home():
    return "Welcome to the SafeBite - AI-powered Allergen Detection API!"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the input data from the POST request
        data = request.get_json()

        # Prepare input data (convert the received JSON into a DataFrame)
        input_data = pd.DataFrame(data, index=[0])

        # Encode the categorical columns
        categorical_columns = input_data.select_dtypes(include=['object']).columns
        input_data_encoded = encoder.transform(input_data[categorical_columns])

        # Concatenate encoded columns back with the numeric columns
        input_data = pd.concat([input_data.drop(categorical_columns, axis=1), input_data_encoded], axis=1)

        # Ensure the DataFrame has the correct column order
        final_input = input_data[['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning', 'Allergens', 'Price ($)', 'Customer rating (Out of 5)']]

        # Make prediction using the trained model
        prediction = model.predict(final_input)

        # Interpret model output
        result = "This product contains allergens" if prediction[0] == 0 else "This product does not contain allergens"

        # Return the result as a JSON response
        response = jsonify(result=result)
        return response
    except Exception as e:
        return jsonify(error=str(e)), 500

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    # Run Flask app
    app.run(port=5000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Public URL: NgrokTunnel: "https://0557-35-229-80-182.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 19:09:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 19:09:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 19:12:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 19:12:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 19:14:44] "GET / HTTP/1.1" 200 -
